In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.3.0


In [4]:
imdb = keras.datasets.imdb

# num_words = 10000 : 가장 많이 등장하는 상위 10000개 단어 선택
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [5]:
print("훈련 샘플 : {}, 레이블 : {}".format(len(train_data), len(train_labels)))

훈련 샘플 : 25000, 레이블 : 25000


In [6]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [7]:
len(train_data[0]), len(train_data[1])

(218, 189)

In [24]:
word_index = {k:(v+3) for k,v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [25]:
decode_review(train_data[0])

'<START> and i of by run total personality thrown up oh taught no tap ? nothing his rather ? was most by viewing after her order <UNK> ? are turn different ? such ? funny after 10 <UNK> nice tell be ? funny cd moves to stories he the cinema ? things her of ? why debut br and i ? candy im already ? i would bad ? of by run he see a the surprisingly ? i to child to ? of shown is <UNK> ? more remember ? ? beautiful ? make ? ? tape challenged of turn no kenneth not ? where ? of he looked ? was never or his ever from was track not ? i ? action with their just ? and written of well <UNK> ? ? them films suggestion a rather ? <UNK> ? scripted ? easily his would by run under as understand john who ? ? <UNK> talented the then get ? women a dvd how on santa has as man ? young ignores is ? role i it off under as turn ? say but reluctant is or his with must into was then ? role up of he producer get ? of book ? of namely think its on a of coincidence br find on'

In [27]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [29]:
len(train_data[0]), len(test_data[0])

(256, 256)

In [30]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [34]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [42]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [43]:
history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=40,
                   batch_size=512,
                   validation_data = (x_val, y_val),
                   verbose=1)

Epoch 1/40
30/30 [==============================] - 0s 15ms/step - loss: 0.6917 - accuracy: 0.5657 - val_loss: 0.6893 - val_accuracy: 0.6909
Epoch 2/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6852 - accuracy: 0.7209 - val_loss: 0.6800 - val_accuracy: 0.7323
Epoch 3/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6708 - accuracy: 0.7627 - val_loss: 0.6619 - val_accuracy: 0.7408
Epoch 4/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6457 - accuracy: 0.7819 - val_loss: 0.6335 - val_accuracy: 0.7566
Epoch 5/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6096 - accuracy: 0.7997 - val_loss: 0.5960 - val_accuracy: 0.7878
Epoch 6/40
30/30 [==============================] - 0s 12ms/step - loss: 0.5649 - accuracy: 0.8219 - val_loss: 0.5526 - val_accuracy: 0.8097
Epoch 7/40
30/30 [==============================] - 0s 12ms/step - loss: 0.5163 - accuracy: 0.8377 - val_loss: 0.5086 - val_accuracy: 0.8219
Epoch 8/40
30

In [44]:
results = model.evaluate(test_data, test_labels, verbose=2)
print(results)

782/782 - 0s - loss: 0.3318 - accuracy: 0.8726
[0.33177122473716736, 0.8726000189781189]
